In [ ]:
import cv2
import numpy as np
import scipy.ndimage
import scipy.stats
import time
import multiprocessing
import ImportNotebook
import GenerateDaugmanTemplateSet
import DaugmanIrisCore
import matplotlib.image as mpimg
import matplotlib.pyplot as plt


mypool =  multiprocessing.Pool()
#？这几个变量的含义
imageWorkingWidth = 150.0
irisRangeLB = 0.16
irisRangeUB = 0.24
irisTemplateSet = []
irisWeights = []
irisRange = []
irisRadiiLog = []

def regulateIrisRange():
    from scipy.stats import norm
    #得到随机序列的期望和标准差
    (mu, sigma) = norm.fit(irisRadiiLog)
    global irisRangeLB
    global irisRangeUB
    irisRangeLB = mu - 2*sigma
    irisRangeUB = mu + 2*sigma
    generateTemplateSet()

def generateTemplateSet():
    global irisTemplateSet
    global irisWeights
    global irisRange
    #?生成的这个数字序列是做啥的？
    #[ 24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.]
    irisRange = np.arange(round(irisRangeLB*imageWorkingWidth),round(irisRangeUB*imageWorkingWidth)+1,1)
#    print '=' *40
#    print(irisRange)
    irisRingWidth = 1
    GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet.pool = mypool
    #?irisTemplateSet是模板集？怎么全是0？
    irisTemplateSet,irisWeights = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet(irisRingWidth,irisRange)
#    print '='*40
#    print irisTemplateSet
    
def EqualizeRGBImage(img):
    #转化成YUV通道
    ycrcb = cv2.cvtColor(img,cv2.COLOR_RGB2YCR_CB)
    #灰度图直方图均衡，第0层是亮度
    ycrcb[:,:,0] = cv2.equalizeHist(ycrcb[:,:,0])
    #再转化回来
    return cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2RGB)


def DaugmanIrisLocalization(img):  
#    cv2.imshow('yglalala1',img)
    #做了亮度的处理，为了更好的找到边界
    imgeq = EqualizeRGBImage(img)
    #cv2.imshow('test1',img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    #cv2.imshow('test2',imglarge)
    #保护边缘的平滑滤波器bilateralFilter
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    #cv2.imshow('test3',imgfiltered)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    cv2.imshow('test4',imggrey)
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
#    print '-' *40
#    print imggrey.shape
#    print [len(irisTemplateSet)]
#    print '-' *40
    
#    for s in xrange(len(irisTemplateSet)):
#        covstackdiffmax,covstackdiffmaxRadius,covstackdiff = DaugmanIrisCore.DaugmanIrisCore(irisTemplateSet[s],imggrey)
#        covstackdiffmaxmaps[:,:,s] = covstackdiffmax
#        covstackdiffmaxradii[:,:,s] = covstackdiffmaxRadius
#        covstackDiffSet.append(covstackdiff)
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
#    print '-' *40
#    print results
    parallelResults = [r.get() for r in results]
#    print '-' *40
#    print parallelResults
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        #？
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]
    ))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
#    print '-'*40
#    print 'eyecenter'
#    print eyecenter
#    print 'eyeiris'
#    print eyeiris
    
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered

def DaugmanIrisLocalizationAnothor(img):  
#    cv2.imshow('yglalala1',img)
    #做了亮度的处理，为了更好的找到边界
    imgeq = EqualizeRGBImage(img)
    #cv2.imshow('test1',img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    #cv2.imshow('test2',imglarge)
    #保护边缘的平滑滤波器bilateralFilter
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    #cv2.imshow('test3',imgfiltered)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    cv2.imshow('test4',imggrey)
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
#    print '-' *40
#    print imggrey.shape
#    print [len(irisTemplateSet)]
#    print '-' *40
    
#    for s in xrange(len(irisTemplateSet)):
#        covstackdiffmax,covstackdiffmaxRadius,covstackdiff = DaugmanIrisCore.DaugmanIrisCore(irisTemplateSet[s],imggrey)
#        covstackdiffmaxmaps[:,:,s] = covstackdiffmax
#        covstackdiffmaxradii[:,:,s] = covstackdiffmaxRadius
#        covstackDiffSet.append(covstackdiff)
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
#    print '-' *40
#    print results
    parallelResults = [r.get() for r in results]
#    print '-' *40
#    print parallelResults
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        #？
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]
    ))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
#    print '-'*40
#    print 'eyecenter'
#    print eyecenter
#    print 'eyeiris'
#    print eyeiris
    
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered
 
def main():
    img = mpimg.imread('/Users/Jiangzp/test.jpg')
    initialize()
    t = time.time()
    for x in range(0,20):
        eyecenter,eyeiris = DaugmanIrisLocalization(img)
    print time.time() -t
    print eyecenter,eyeiris
    
    mypool.close()
    mypool.join()

    plt.imshow(img)
    plt.show()

def localizeIrisAndShow(img):
    initialize()
    eyecenter,eyeiris = DaugmanIrisLocalization(img)
    cv2.circle(img,tuple(eyecenter),int(eyeiris),(0,0,255))
    plt.imshow(img)
    plt.show()
